# 단어생성기

In [1]:
import PIL.Image as Image
import tqdm
import easydict
import random

import argparse
import os
import numpy as np
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt

PHD_PATH = './phd08'



def font_start_checker(line):
    if not line.strip():
        return True
    else:
        return False


def txt_to_npy(all_file_count, index, data, labels,
               file_full_path, width, height, sigma, is_one_hot):
    index -= 1
    with open(file_full_path, 'r') as lines:
        font_counter = 0
        not_data_checker = 0
        font_array = []
        real_data_counter = 0
        for line in lines:
            if font_start_checker(line):  # endl
                not_data_checker = 0
                font_counter += 1
                real_data_counter = 0

                if len(font_array) == 0:
                    continue
                # data
                font_blurred_array = font_array*255
                temp = Image.fromarray(font_blurred_array)
                temp = temp.resize((width,height))
                
                font_blurred_array = np.array(temp)
                
#                 plt.imshow(temp, cmap=plt.cm.gray_r, interpolation = "nearest")

                data.append(font_blurred_array)

                # labels
                if is_one_hot == True:
                    label = np.zeros(shape=(all_file_count))
                    label[index] = 1
                else:
                    label = index
                labels.append(label)

                font_array = []

                continue
            else:  # not endl
                not_data_checker += 1
                if not_data_checker == 1:  # font name
                    continue
                elif not_data_checker == 2:  # font size
                    arr_height = int(line.split(' ')[0])
                    arr_width = int(line.split(' ')[1])
                    font_array = np.zeros(shape=(arr_height, arr_width))
                    continue
                else:  # this is real data
                    font_array[real_data_counter] = list(map(int, line.strip()))
                    real_data_counter += 1
                    continue
    return data, labels




def convert_text(word):
#     args = parse_args()
    args = easydict.EasyDict({ 
        "data_dir": './phd08',
        "one_hot": False,
        "width": 56,
        "height": 56,
        "gaussian_sigma": 3,
        "batch_size": 2

    })
    
    if args is None:
        exit()

    # 존재유무 체크
    if not os.path.exists(args.data_dir):
        print("ERROR::" + args.data_dir, " 는 존재하지 않는 폴더입니다.")
        exit()

    save_dir = 'phd08_npy_results'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    data = []
    labels = []
    # 쪼개서 저장할 사이즈
    batch_size = args.batch_size

    print(args)
    # 전체 파일 갯수 체크
    all_file_count = 0
    for _, _, files in os.walk(args.data_dir):
        for file in files:
            if file[0] == '.':
                continue
            all_file_count += 1

    for _, _, files in os.walk(args.data_dir):
        index = 0
        for alpha in word:
            for file in tqdm.tqdm_notebook(files):
                if(file != alpha+'.txt'):
                    continue
                if file[0] == '.':
                    continue
                index += 1
                print("INFO:: converting " + file + "...")
                data, labels = txt_to_npy(all_file_count, index, data, labels,
                                          args.data_dir + '/' + file, args.width, args.height, args.gaussian_sigma,
                                          args.one_hot)
                print(index)
#                 if index % batch_size == 0:
                data = np.array(data).astype('int')
                labels = np.array(labels)
                out_data = save_dir + '/phd08_data_'+ alpha
                out_labels = save_dir + '/phd08_labels_'+ alpha
                np.save(out_data, data)
                np.save(out_labels, labels)
                data = []
                labels = []
                print("  FILE_SAVED:: filename : " + save_dir + '/phd08_data_' + str(int(index / batch_size)))

        print("INFO:: all files converted to npy file, results in phd08_npy_results")

C:\Users\UOK\anaconda3\envs\tensor2\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\UOK\anaconda3\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\UOK\anaconda3\envs\tensor2\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


### 단일글자 리스트 생성

In [2]:
file_list = os.listdir(PHD_PATH)
word_list = []
for file in file_list:
    file_name = file.split('.')[0]
    word_list.append(file_name)

In [3]:
for word in word_list:
    convert_text(word)

{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


C:\Users\UOK\anaconda3\envs\tensor2\lib\site-packages\ipykernel_launcher.py:120: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


INFO:: converting 가.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 각.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 간.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


...

INFO:: converting 힐.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 힘.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 힙.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 힛.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
{'data_dir': './phd08', 'one_hot': False, 'width': 56, 'height': 56, 'gaussian_sigma': 3, 'batch_size': 2}


INFO:: converting 힝.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results


In [5]:
convert_text("나")

{'data_dir': './phd08', 'one_hot': False, 'width': 28, 'height': 28, 'gaussian_sigma': 3, 'batch_size': 2}


C:\Users\UOK\anaconda3\envs\tensor2\lib\site-packages\ipykernel_launcher.py:119: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


INFO:: converting 나.txt...
1
  FILE_SAVED:: filename : phd08_npy_results/phd08_data_0

INFO:: all files converted to npy file, results in phd08_npy_results
